In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
from tabulate import tabulate
from fuzzywuzzy import fuzz, process

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soleadify-dataset/google dataset.csv
/kaggle/input/soleadify-dataset/website dataset.csv
/kaggle/input/soleadify-dataset/facebook dataset.csv


**<font size="6"> DATASETS MERGING </font>** 
</br>
</br>
</br>
We have three datasets containing information about the same companies, extracted from three different places:
* Facebook 
* Google
* Companies' websites 

The final goal of this project is to create a fourth dataset which must be obtained by joining the other three datasets, in order to get a high accuracy (as high as possible) for the following common columns: Category, Address(Country, Region, City), Phone, Company names. 



First things first, I will clean the three datasets.

**<font size="4"> WEBSITE DATASET:</font>**

In [2]:
web_ds = pd.read_csv('/kaggle/input/soleadify-dataset/website dataset.csv', error_bad_lines=False, low_memory=False)
web_ds.head(15)
print('no. of rows: ',web_ds.shape[0])
print('no. of columns: ', web_ds.shape[1])
web_ds.head(15)

no. of rows:  71917
no. of columns:  17


,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores,NaN,NaN,NaN,NaN,NaN,NaN
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers,NaN,NaN,NaN,NaN,NaN,NaN
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers,NaN,NaN,NaN,NaN,NaN,NaN
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting,NaN,NaN,NaN,NaN,NaN,NaN
5,ironcrow.ca,ca,en,NaN,calgary,canada,alberta,14032878770,Iron Crow,ca,Furniture Stores,NaN,NaN,NaN,NaN,NaN,NaN
6,springboarddm.com,com,en,The Hershey Company,mississauga,canada,ontario,19053690553,SpringBoard Data Management,com,Data Solutions,NaN,NaN,NaN,NaN,NaN,NaN
7,stoneandtilesrus.com,com,en,NaN,brampton,canada,ontario,19054940660,STONE AND TILES R US,com,Tile Store,NaN,NaN,NaN,NaN,NaN,NaN
8,mdaccpap.com,com,en,MDAC CPAP INC,thunder bay,canada,ontario,18076834405,MDAC CPAP,com,Medical Supply Manufacturers,NaN,NaN,NaN,NaN,NaN,NaN
9,micacchi.ca,ca,en,NaN,toronto,canada,ontario,16477257799,Micacchi Architecture,ca,Architects & Architectural Services,NaN,NaN,NaN,NaN,NaN,NaN


Since there are several columns which consist of NaN values, I eliminated them. 

In [3]:
web_ds.drop(web_ds.iloc[:, 11:], inplace = True, axis = 1)
# web_ds.head(15)

Since we want the dataset to be organised by the company names, I will check the percentage of unique company names.

In [4]:
print('Unqiue company names in the dataset: ',np.round(len(web_ds['legal_name'].unique())/web_ds.shape[0]*100, 2),'%')

Unqiue company names in the dataset:  42.53 %


I noticed that less than half of the dataset consists of unique company names. I explored the dataset using Excel and noticed that there are many company names assigned to a NaN value. This, I checked the percentage of NaN values in the company names.

In [5]:
print('NaN values percentage in company names: ',np.round(web_ds['legal_name'].isnull().sum()/web_ds.shape[0]*100, 2),'%')

NaN values percentage in company names:  55.61 %


Since more than half of the companies have their names missing, I will eliminate them, since the company name is the most important detail so we can't work with missing values.

In [6]:
# web_ds = web_ds.set_index("legal_name")
web_ds.dropna(subset=['legal_name'], inplace=True)
print('NaN values percentage in company names: ',np.round(web_ds['legal_name'].isnull().sum()/web_ds.shape[0]*100, 2),'%')

NaN values percentage in company names:  0.0 %


Now that there are no NaN values left, I will eliminate the duplicate names and replace any NaN value with blank spaces. I will continue analysing the dataset and the unique values for each column.

In [7]:
web_ds = web_ds.drop_duplicates(subset=['legal_name'])
web_ds = web_ds.replace(np.nan, '', regex=True)
for column in web_ds.columns:
    print(column, ': unique values percentage :',np.round(len(web_ds[column].unique())/web_ds.shape[0]*100, 2),'%')

root_domain : unique values percentage : 100.0 %
domain_suffix : unique values percentage : 1.03 %
language : unique values percentage : 0.05 %
legal_name : unique values percentage : 100.0 %
main_city : unique values percentage : 15.82 %
main_country : unique values percentage : 0.59 %
main_region : unique values percentage : 2.39 %
phone : unique values percentage : 91.36 %
site_name : unique values percentage : 96.87 %
tld : unique values percentage : 0.6 %
s_category : unique values percentage : 1.84 %


In [8]:
web_ds['phone'] = web_ds['phone'].astype(str)
# web_ds = web_ds.transform(lambda x: x.str.lower())
web_ds.reset_index()
web_ds.head(15)

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,,Canada,,,Destinate Group,ca,Business Consulting
6,springboarddm.com,com,en,The Hershey Company,mississauga,canada,ontario,19053690553,SpringBoard Data Management,com,Data Solutions
8,mdaccpap.com,com,en,MDAC CPAP INC,thunder bay,canada,ontario,18076834405,MDAC CPAP,com,Medical Supply Manufacturers
11,libertelightdance.com,com,en,"LIBERTE LIGHT DANCE, SCHOOL OF DANCE INC.",moose jaw,canada,saskatchewan,13069900067,Liberte Light Dance,com,Dance Schools
13,unitysodfarm.com,com,en,Unity Sod Farm Ltd.,,,,16133893437,Unity Sod Farm,com,Farms & Agriculture Production
14,cdncoin.com,com,en,HUDSON'S BAY COMPANY,richmond hill,canada,ontario,18882362646,Royal Canadian Mint,com,Watches & Jewelry Stores
15,parlourcoffee.ca,ca,en,Parlour Coffee Ltd.,winnipeg,canada,manitoba,12049427679,Parlour Coffee,ca,Coffee & Tea Shops
16,freshdishcatering.ca,ca,en,Maduro Coffee Company,saskatoon,canada,saskatchewan,13062625586,Fresh Dish Catering,ca,Catering & Delivery


I will do the same thing for the other two datasets. </br>
</br>
</br>
**<font size="4"> FACEBOOK DATASET: </font>**

In [9]:
fb_ds = pd.read_csv('/kaggle/input/soleadify-dataset/facebook dataset.csv', error_bad_lines=False, low_memory=False, warn_bad_lines=False)
print('no. of rows: ',fb_ds.shape[0])
print('no. of columns: ', fb_ds.shape[1])

no. of rows:  71167
no. of columns:  16


I explored this dataset using Excel and noticed 72000 rows. Some rows where lost due to the reading errors which I couldn't solve. </br>
I also converted the phone number values from float to integer. 

In [10]:
print('Unqiue company names in the dataset: ',np.round(len(fb_ds['name'].unique())/fb_ds.shape[0]*100, 2),'%')
print('NaN values percentage in company names: ',np.round(fb_ds['name'].isnull().sum()/fb_ds.shape[0]*100, 2),'%')

Unqiue company names in the dataset:  99.76 %
NaN values percentage in company names:  0.0 %


In [11]:
fb_ds = fb_ds.drop_duplicates(subset=['name'])
# print('Unqiue company names in the dataset: ',np.round(len(fb_ds['name'].unique())/fb_ds.shape[0]*100, 2),'%')
# print(fb_ds)
for column in fb_ds.columns:
    print(column, ': unique values percentage :',np.round(len(fb_ds[column].unique())/fb_ds.shape[0]*100, 2),'%')

domain : unique values percentage : 100.0 %
address : unique values percentage : 79.34 %
categories : unique values percentage : 14.17 %
city : unique values percentage : 8.89 %
country_code : unique values percentage : 0.18 %
country_name : unique values percentage : 0.16 %
description : unique values percentage : 37.72 %
email : unique values percentage : 27.79 %
link : unique values percentage : 100.0 %
name : unique values percentage : 100.0 %
page_type : unique values percentage : 0.01 %
phone : unique values percentage : 62.25 %
phone_country_code : unique values percentage : 0.14 %
region_code : unique values percentage : 0.61 %
region_name : unique values percentage : 0.96 %
zip_code : unique values percentage : 36.5 %


In [12]:
fb_ds['phone'] = fb_ds['phone'].replace('',0,regex=True)
fb_ds['phone'] = fb_ds['phone'].fillna(0).astype(int)
fb_ds['phone'] = fb_ds['phone'].replace(0,'',regex=True)
fb_ds['phone'] = fb_ds['phone'].astype(str)

Since I noticed that this dataset features more than one category for most of the companies, I inserted these categories in lists. 

In [13]:
fb_ds = fb_ds.replace(np.nan, '', regex=True)
fb_ds['categories'] = fb_ds['categories'].transform(lambda x: x.split('|'))
fb_ds.head(15)

,domain,address,categories,city,country_code,country_name,description,email,link,name,page_type,phone,phone_country_code,region_code,region_name,zip_code
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",[],calvisson,fr,france,,,https://euro-hygiene-34.fr,Euro Hygiène,LocalBusiness,,,occ,occitanie,30420
1,lakesidehomeservices.ca,,"[Appliance Repair & Maintenance, Home Builders...",,ca,,,,https://lakesidehomeservices.ca,Lakeside Home Services,LocalBusiness,16138318840,ca,,,
2,rossiterboats.com,,"[Boats & Yachts Dealers, Boat Tours & Cruises]",,,,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,Rossiter Boats,LocalBusiness,,,,,
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...","[Architects & Architectural Services, Other En...",vancouver,ca,canada,,office@caa-arcitecture.com,http://caa-architecture.com,Chandler Associates Architecture Inc.,LocalBusiness,16046873390,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",[],winnipeg,ca,canada,,,https://apexsurety.ca,Apex Surety & Insurance Ltd.,LocalBusiness,12045602508,ca,mb,manitoba,r3l 2t7
5,collins-cc.edu,,"[Other schools, High Schools, Community Center...",,,,,,https://collins-cc.edu,Collins Career Technical Center,LocalBusiness,,,,,
6,dovercheese.com,"307 main street, n0a 1n0, port dover, on, cana...",[],port dover,ca,canada,The Dover Cheese Shop is a food lover’s paradi...,info@dovercheese.com,http://dovercheese.com,The Dover Cheese Shop,LocalBusiness,,,on,ontario,n0a 1n0
7,alphascientific.ca,"8623 granville street unit 143, v6p 5a2, vanco...",[Medical Supply Manufacturers],vancouver,ca,canada,www.alphascientific.ca Alpha Scientific Repair...,sales@alphascientific.ca,http://alphascientific.ca,Alpha Scientific Repair Services Ltd.,LocalBusiness,16046903380,ca,bc,british columbia,v6p 5a2
8,modernmama.com,,[Book Publisher],,,,,,https://modernmama.com/edmonton,Modern Mama Edmonton,Person,,,,,
9,unitysodfarm.com,"2989 unity road, k0h 1m0, kingston, on, canada...","[Agriculture & Farm Associations, Garden Equip...",,ca,,"Kingston's local supplier of quality sod, seed...",info@unitysodfarm.com,http://unitysodfarm.com,Unity Sod Farm Ltd,LocalBusiness,16133893437,ca,,,


**<font size="4"> GOOGLE DATASET: </font>**

In [14]:
ggl_ds = pd.read_csv('/kaggle/input/soleadify-dataset/google dataset.csv', error_bad_lines=False, low_memory=False, warn_bad_lines=False)
print('no. of rows: ',ggl_ds.shape[0])
print('no. of columns: ', ggl_ds.shape[1])

no. of rows:  346925
no. of columns:  15


In [15]:
print('Unqiue company names in the dataset: ',np.round(len(ggl_ds['name'].unique())/ggl_ds.shape[0]*100, 2),'%')
print('NaN values percentage in company names: ',np.round(ggl_ds['name'].isnull().sum()/ggl_ds.shape[0]*100, 2),'%')

Unqiue company names in the dataset:  99.27 %
NaN values percentage in company names:  0.01 %


In [16]:
ggl_ds.dropna(subset=['name'], inplace=True)
print('NaN values percentage in company names: ',np.round(ggl_ds['name'].isnull().sum()/ggl_ds.shape[0]*100, 2),'%')

NaN values percentage in company names:  0.0 %


I removed the '+' symbol in the phone numbers so they should correspond to the phone numbers in the other datasets. 

In [17]:
ggl_ds['phone'] = ggl_ds['phone'].transform(lambda x: x.str.replace('+', '',regex=True))
ggl_ds['phone'] = ggl_ds['phone'].transform(lambda x: x.str.strip())
ggl_ds = ggl_ds.replace(np.nan, '', regex=True)
ggl_ds['phone'] = ggl_ds['phone'].astype(str)

In [18]:
ggl_ds.head(15)

,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain
0,"28 Central Coast Hwy, West Gosford NSW 2250, A...",Fabric-Based Home Goods,gosford,au,australia,Spotlight West Gosford,61243355946,au,"West Gosford NSW, Australia",+61 2 4335 5946,nsw,new south wales,"4.1 (766) · Craft store West Gosford NSW, Aust...",2250,spotlightstores.com
1,"400 Scott St, St. Catharines, ON L2M 3W2, Canada",Book Stores,st. catharines,ca,canada,Heritage Christian Book Store,19059374553,ca,400 Scott St · In Grantham Plaza,+1 905-937-4553,on,ontario,4.7 (100) · Book store 400 Scott St · In Grant...,l2m 3w2,bookmanager.com
2,"191 Pleasant St, Yarmouth, NS B5A 2J9, Canada",Other Building Material Retailers,yarmouth,ca,canada,Pleasant Timber Mart,19027429181,ca,"7+ years in business · Yarmouth, NS, Canada",+1 902-742-9181,ns,nova scotia,4.7 (40) · Building materials store 7+ years i...,b5a 2j9,timbermart.ca
3,"11040 Santa Monica Blvd Suite 370, Los Angeles...",Plastic Surgery Clinics,los angeles,us,united states,Skin Specifics Medical Spa West LA,18184268353,us,7+ years in business · 11040 Santa Monica Blvd...,+1 818-426-8353,ca,california,4.3 (15) · Medical spa 7+ years in business · ...,90025,linktr.ee
4,,Catering & Delivery,,,,Lakay express,,,,,,,5.0 (1) · Mobile caterer Open ⋅ Closes 9PM,,linktr.ee
5,"Ferndale, MI, United States",Pubs & Bars,ferndale,us,united states,B. Nektar Meadery - Taproom & Headquarters,13137446323,us,"Ferndale, MI, United States",+1 313-744-6323,mi,michigan,"4.8 (296) · $$ · Bar Ferndale, MI, United Stat...",,linktr.ee
6,"321 Kent St, Sydney NSW 2000, Australia","Insurance - Agents, Carriers & Brokers",sydney,au,australia,icare NSW,,,"Sydney NSW, Australia",,nsw,new south wales,"2.3 (187) · Insurance company Sydney NSW, Aust...",2000,icare.nsw.gov.au
7,"55 S Cleveland Ave, Westerville, OH 43081",Preschools & Kindergartens,westerville,us,united states,South Cleveland Avenue KinderCare,16148990026,,"7+ years in business · Westerville, OH",(614) 899-0026,oh,ohio,3.6 (20) · Preschool 7+ years in business · We...,43081,kindercare.com
8,"4050 S Torrey Pines Dr, Las Vegas, NV 89103, U...",Preschools & Kindergartens,las vegas,us,united states,Torrey Pines KinderCare,17023670822,us,"7+ years in business · Las Vegas, NV, United S...",+1 702-367-0822,nv,nevada,3.8 (11) · Preschool 7+ years in business · La...,89103,kindercare.com
9,"7+ years in business · St Paul, MN, United States",Preschools & Kindergartens,,us,,Meadowlands KinderCare,16514296316,us,"7+ years in business · St Paul, MN, United States",(651) 429-6316,,,4.8 (9) · Kindergarten 1000 Meadowlands Dr Clo...,,kindercare.com


**<font size="4"> MERGING THE THREE DATASETS </font>**
</br>
</br>
</br>
Now that we finally have all the datasets containing unique company names, we can further modify the company names by removing abbreviations like 'LTD' or 'INC' so we'll know that a company which is called, for example, 'Facebook Inc.' is the same one with 'Facebook'. 

In [19]:
abbreviations = ['ltd','inc','llc','org','gmbh','corp','mfg','mfrs','ltd.','inc.','llc.','org.','gmbh.','corp.','mfg.','mfrs.',',','-','.']

web_ds['legal_name'] = web_ds['legal_name'].transform(lambda x: x.str.lower())
web_ds['legal_name'] = web_ds['legal_name'].apply(lambda x: ' '.join([word for word in x.split() if word not in abbreviations]))

fb_ds['name'] = fb_ds['name'].transform(lambda x: x.str.lower())
fb_ds['name'] = fb_ds['name'].apply(lambda x: ' '.join([word for word in x.split() if word not in abbreviations]))

ggl_ds['name'] = ggl_ds['name'].transform(lambda x: x.str.lower())
ggl_ds['name'] = ggl_ds['name'].apply(lambda x: ' '.join([word for word in x.split() if word not in abbreviations]))

I will now determine the percentage of common company names in the three datasets.

In [20]:
intersect = np.intersect1d(web_ds['legal_name'], fb_ds['name'])
intersect = np.intersect1d(ggl_ds['name'], intersect)
print('common companies in all three datasets: ',len(intersect))
print('percentage of common companies in the website dataset: ', np.round(len(intersect)/len(web_ds['legal_name'])*100, 2),'%')
print('percentage of common companies in the Facebook dataset: ', np.round(len(intersect)/len(fb_ds['name'])*100, 2), '%')
print('percentage of common companies in the Google dataset:', np.round(len(intersect)/len(ggl_ds['name'])*100, 2), '%')

common companies in all three datasets:  11637
percentage of common companies in the website dataset:  38.05 %
percentage of common companies in the Facebook dataset:  16.39 %
percentage of common companies in the Google dataset: 3.35 %


Looks like most of the common companies are found in the website dataset, which is the smallest one. We'll get a pretty small merged dataset. I will rename the columns so there won't be any confusion, since in the Google and Facebook datasets the company names correspond to the 'name' column, the country names correspond to the 'country_name' column etc. 

In [21]:
web_ds.columns = ['web_root_domain', 'web_domain_suffix', 'web_language', 'web_legal_name', 'web_main_city',
       'web_main_country', 'web_main_region', 'web_phone', 'web_site_name', 'web_tld',
       'web_category']
fb_ds.columns = ['fb_domain', 'fb_address', 'fb_categories', 'fb_city', 'fb_country_code',
       'fb_country_name', 'fb_description', 'fb_email', 'fb_link', 'fb_name', 'fb_page_type',
       'fb_phone', 'fb_phone_country_code', 'fb_region_code', 'fb_region_name',
       'fb_zip_code']
ggl_ds.columns = ['ggl_address', 'ggl_category', 'ggl_city', 'ggl_country_code', 'ggl_country_name', 'ggl_name',
       'ggl_phone', 'ggl_phone_country_code', 'ggl_raw_address', 'ggl_raw_phone',
       'ggl_region_code', 'ggl_region_name', 'ggl_text', 'ggl_zip_code', 'ggl_domain']

I will now merge the three datasets.

In [22]:
ds = fb_ds.merge(ggl_ds, left_on="fb_name", right_on="ggl_name")
ds = ds.merge(web_ds, left_on='ggl_name', right_on='web_legal_name')
ds = ds.reset_index()
ds.head(15)

,index,fb_domain,fb_address,fb_categories,fb_city,fb_country_code,fb_country_name,fb_description,fb_email,fb_link,...,web_domain_suffix,web_language,web_legal_name,web_main_city,web_main_country,web_main_region,web_phone,web_site_name,web_tld,web_category
0,0,lakesidehomeservices.ca,,"[Appliance Repair & Maintenance, Home Builders...",,ca,,,,https://lakesidehomeservices.ca,...,ca,en,lakeside home services,stittsville,canada,ontario,16138318840,Lakeside Home Services,ca,Appliance Repair & Maintenance
1,1,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...","[Architects & Architectural Services, Other En...",vancouver,ca,canada,,office@caa-arcitecture.com,http://caa-architecture.com,...,com,en,chandler associates architecture,vancouver,canada,british columbia,16046873325,Chandler,com,Architects & Architectural Services
2,2,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",[],winnipeg,ca,canada,,,https://apexsurety.ca,...,ca,en,apex surety & insurance,winnipeg,canada,manitoba,12045602508,INSURANCEAPEX,ca,Liability Insurance Agency
3,3,alphascientific.ca,"8623 granville street unit 143, v6p 5a2, vanco...",[Medical Supply Manufacturers],vancouver,ca,canada,www.alphascientific.ca Alpha Scientific Repair...,sales@alphascientific.ca,http://alphascientific.ca,...,ca,en,alpha scientific repair services,vancouver,canada,british columbia,16046903380,Alpha Scientific,ca,Medical Supply Manufacturers
4,4,unitysodfarm.com,"2989 unity road, k0h 1m0, kingston, on, canada...","[Agriculture & Farm Associations, Garden Equip...",,ca,,"Kingston's local supplier of quality sod, seed...",info@unitysodfarm.com,http://unitysodfarm.com,...,com,en,unity sod farm,,,,16133893437,Unity Sod Farm,com,Farms & Agriculture Production
5,5,vintageautogp.com,"34007 twp rd 713a, county of gp no 1, t8x 4b7,...","[Auto Services, Auto Parts Store, Automobile D...",grande prairie,ca,canada,Vintage Auto is a company that rebuilds vintag...,elesko@vintageautogp.com,http://vintageautogp.com,...,com,en,vintage auto restoration services,,,,17808148233,Vintage Auto Restoration Services,com,Auto Restoration Service
6,6,shariffitness.com,"1050 belcourt st, l1x 0g3, pickering, on, cana...","[Personal Trainer, Fitness & Health Centers, F...",pickering,ca,canada,,,https://shariffitness.com,...,com,en,sharif fitness,pickering,canada,ontario,16474956405,SHARIF FITNESS,com,Martial Arts Gyms
7,7,powerstrokeltd.com,"18-34 wrangler place se, t1x 0l7, calgary, ab,...",[Towing Wrecking & Roadside Assistance],calgary,ca,canada,,,https://powerstrokeltd.com,...,com,en,power stroke truck centre,,,,14035707782,Power Stroke Truck Centre,com,Auto Services
8,8,kingswaytransmission.com,"195 westney road south, unit 5, l1s 2c9, ajax,...","[Auto Services, Auto Parts Store, Automobile D...",ajax,ca,canada,Kingsway Transmission has been providing excep...,kingswaytransmission@gmail.com,http://kingswaytransmission.com,...,com,en,kingsway transmission,ajax,canada,ontario,19054288940,Kingsway Transmission,com,Auto Services
9,9,parksiderentals.ca,"birds hill park hwy 206, r0e 0k0, winnipeg, mb...","[Boat Rental & Chartering Services, Boat Tours...",winnipeg,ca,canada,All rentals come with a trailer so you can go ...,,http://parksiderentals.ca,...,ca,en,parkside rentals,dugald,canada,manitoba,12042325434,Parkside Rentals,ca,Boat Rental & Chartering Services


The columns which we are interesed in are: category, region, country, city, phone, compnay name. Since we already have 100% accuracy in the company name column, we well check the other ones. **The Facebook dataset features multiple categories for almost every company, so if any of the categories is present in the other datasets, it will count as a match.**


In [23]:
ds['company_match'] = ((ds['web_legal_name'] == ds['ggl_name'])&(ds['ggl_name'] == ds['fb_name']))


ds['country_match'] = ((ds['web_main_country'] == ds['fb_country_name'])&(ds['fb_country_name'] == ds['ggl_country_name']))

ds['region_match'] = ((ds['web_main_region'] == ds['fb_region_name'])&(ds['fb_region_name'] == ds['ggl_region_name']))

ds['phone_match'] = ((ds['web_phone'] == ds['fb_phone'])&(ds['fb_phone'] == ds['ggl_phone']))

ds['city_match'] = ((ds['web_main_city'] == ds['fb_city']) & (ds['fb_city'] == ds['ggl_city']))

true = 0
for i in range(ds.shape[0]):
    row = ds.iloc[i]
    for item in row['fb_categories']:
        if row['ggl_category'] == item and row['web_category'] == item:
            true += 1


data = [['Accuracy', (ds['company_match'].value_counts()/ds.shape[0]*100).iloc[0], (np.round(ds['country_match'].value_counts()/ds.shape[0]*100,2)).iloc[0],
     (np.round(ds['region_match'].value_counts()/ds.shape[0]*100,2)).iloc[0], (np.round(ds['city_match'].value_counts()/ds.shape[0]*100,2)).iloc[0],
         (np.round(ds['phone_match'].value_counts()/ds.shape[0]*100,2)).iloc[0], np.round(true/ds.shape[0]*100,2)]]
table = tabulate(data, headers=['','Company Match', 'Country Match', 'Region Match', 'City Match', 'Phone Match', 'Category Match'], tablefmt='orgtbl')
print(table)
            


|          |   Company Match |   Country Match |   Region Match |   City Match |   Phone Match |   Category Match |
|----------+-----------------+-----------------+----------------+--------------+---------------+------------------|
| Accuracy |             100 |           55.03 |          52.88 |        56.12 |         50.31 |            20.26 |


This doesn't look good. We get around 50% accuracy for all the columns expect for the category column, where the accuracy is even lower, and the company name column. That's mainly becuase there are missing values in the columns. For example, if we find out from the *'fb_country_name'* column that a company is located in France, but the '*ggl_country_name*' corresponding column has a missing value, it will count as a non-match. 
* First, we merge the Facebook and Google datasets, replace any missing value with the correct value to get a higher accuracy. 
* The resulting dataset will be merged with the website dataset, repeating these steps.


In [24]:
ds.to_csv('first_complete_dataset.csv')

**<font size="4"> MERGING THE FACEBOOK AND GOOGLE DATASETS </font>**

In [25]:
intersect = np.intersect1d(ggl_ds['ggl_name'], fb_ds['fb_name'])
print('common companies in all three datasets: ',len(intersect))
print('percentage of common companies in the Facebook dataset: ', np.round(len(intersect)/len(fb_ds['fb_name'])*100, 2), '%')
print('percentage of common companies in the Google dataset:', np.round(len(intersect)/len(ggl_ds['ggl_name'])*100, 2), '%')

common companies in all three datasets:  40900
percentage of common companies in the Facebook dataset:  57.61 %
percentage of common companies in the Google dataset: 11.79 %


We can see that more than half of the Facebook companies can be found in the Google dataset, and moreover, we get a 4 times bigger dataset than the previos one which contains all the three datasets. 

In [26]:
ds_fb_ggl = fb_ds.merge(ggl_ds, left_on="fb_name", right_on="ggl_name")
ds_fb_ggl.head(15)

,fb_domain,fb_address,fb_categories,fb_city,fb_country_code,fb_country_name,fb_description,fb_email,fb_link,fb_name,...,ggl_name,ggl_phone,ggl_phone_country_code,ggl_raw_address,ggl_raw_phone,ggl_region_code,ggl_region_name,ggl_text,ggl_zip_code,ggl_domain
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",[],calvisson,fr,france,,,https://euro-hygiene-34.fr,euro hygiène,...,euro hygiène,,,"3+ years in business · Calvisson, France",+33 4 66 04 02 00,occ,occitanie,4.7 (3) · Pest control service 3+ years in bus...,,euro-hygiene-34.fr
1,lakesidehomeservices.ca,,"[Appliance Repair & Maintenance, Home Builders...",,ca,,,,https://lakesidehomeservices.ca,lakeside home services,...,lakeside home services,16138318840,ca,,+1 613-831-8840,,,5.0 (2) · Appliance repair service Closed ⋅ Op...,,lakesidehomeservices.ca
2,rossiterboats.com,,"[Boats & Yachts Dealers, Boat Tours & Cruises]",,,,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,rossiter boats,...,rossiter boats,15199861203,ca,"Markdale, ON, Canada",+1 519-986-1203,on,ontario,"4.5 (4) · Boat builders Markdale, ON, Canada O...",,rossiterboats.com
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...","[Architects & Architectural Services, Other En...",vancouver,ca,canada,,office@caa-arcitecture.com,http://caa-architecture.com,chandler associates architecture,...,chandler associates architecture,16046873390,ca,"5+ years in business · Vancouver, BC, Canada ·...",+1 604-687-3390,bc,british columbia,4.5 (2) · Architect 5+ years in business · Van...,v6c 2g8,caa-architecture.com
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",[],winnipeg,ca,canada,,,https://apexsurety.ca,apex surety & insurance,...,apex surety & insurance,12045602508,ca,4 Donald St #3,+1 204-560-2508,mb,manitoba,No reviews · Insurance broker 4 Donald St #3 O...,r3l 2t7,apexsurety.ca
5,collins-cc.edu,,"[Other schools, High Schools, Community Center...",,,,,,https://collins-cc.edu,collins career technical center,...,collins career technical center,17408676641,us,"Chesapeake, OH, United States",+1 740-867-6641,,,"4.3 (16) · Technical school Chesapeake, OH, Un...",,collins-cc.edu
6,dovercheese.com,"307 main street, n0a 1n0, port dover, on, cana...",[],port dover,ca,canada,The Dover Cheese Shop is a food lover’s paradi...,info@dovercheese.com,http://dovercheese.com,the dover cheese shop,...,the dover cheese shop,15197181201,ca,"Port Dover, ON, Canada · +1 519-718-1201",+1 519-718-1201,on,ontario,"Cheese shop Port Dover, ON, Canada · +1 519-71...",n0a 1n0,dovercheese.com
7,alphascientific.ca,"8623 granville street unit 143, v6p 5a2, vanco...",[Medical Supply Manufacturers],vancouver,ca,canada,www.alphascientific.ca Alpha Scientific Repair...,sales@alphascientific.ca,http://alphascientific.ca,alpha scientific repair services,...,alpha scientific repair services,16046903380,ca,"Vancouver, BC, Canada",+1 604-690-3380,bc,british columbia,No reviews · Medical equipment supplier Vancou...,v6p 5a3,alphascientific.ca
8,unitysodfarm.com,"2989 unity road, k0h 1m0, kingston, on, canada...","[Agriculture & Farm Associations, Garden Equip...",,ca,,"Kingston's local supplier of quality sod, seed...",info@unitysodfarm.com,http://unitysodfarm.com,unity sod farm,...,unity sod farm,16133893437,ca,"Elginburg, ON, Canada",+1 613-389-3437,,,"3.9 (18) · Farm Elginburg, ON, Canada Closed ⋅...",,unitysodfarm.com
9,healthability.org.au,"917 main road, 3095, eltham, vic, australia, v...","[Security Guards & Patrol Services, Dentists &...",eltham,au,australia,,,https://healthability.org.au,healthability,...,healthability,61394309100,au,"Eltham VIC, Australia",+61 3 9430 9100,vic,victoria,"4.2 (41) · Community health centre Eltham VIC,...",3095,healthability.org.au


In [27]:
ds_fb_ggl['company_match'] = (ds_fb_ggl['ggl_name'] == ds_fb_ggl['fb_name'])

ds_fb_ggl['country_match'] = (ds_fb_ggl['fb_country_name'] == ds_fb_ggl['ggl_country_name'])

ds_fb_ggl['region_match'] =(ds_fb_ggl['fb_region_name'] == ds_fb_ggl['ggl_region_name'])

ds_fb_ggl['phone_match'] = (ds_fb_ggl['fb_phone'] == ds_fb_ggl['ggl_phone'])

ds_fb_ggl['city_match'] = (ds_fb_ggl['fb_city'] == ds_fb_ggl['ggl_city'])

true = 0
for i in range(ds_fb_ggl.shape[0]):
    row = ds_fb_ggl.iloc[i]
    for item in row['fb_categories']:
        if row['ggl_category'] == item:
            true += 1
            
data = [['Accuracy', (ds_fb_ggl['company_match'].value_counts()/ds_fb_ggl.shape[0]*100).iloc[0], 
         (np.round(ds_fb_ggl['country_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0],
         (np.round(ds_fb_ggl['region_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
         (np.round(ds_fb_ggl['city_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
         (np.round(ds_fb_ggl['phone_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
          np.round(true/ds_fb_ggl.shape[0]*100,2)]]
table = tabulate(data, headers=['','Company Match', 'Country Match', 'Region Match', 'City Match', 'Phone Match', 'Category Match'], tablefmt='orgtbl')
print(table)

|          |   Company Match |   Country Match |   Region Match |   City Match |   Phone Match |   Category Match |
|----------+-----------------+-----------------+----------------+--------------+---------------+------------------|
| Accuracy |             100 |           66.06 |          64.35 |        58.26 |         58.08 |            35.13 |


Ok, this looks a bit better than the previous dataset, but the accuracy is still  low for all the columns. I will now fill the missing values. 

In [28]:
def fill_missing_values(fb_column, ggl_column):
    ds_fb_ggl[fb_column] = ds_fb_ggl[fb_column].replace('',np.nan, regex=True)
    ds_fb_ggl[fb_column].fillna(ds_fb_ggl[ggl_column], inplace=True)
    ds_fb_ggl[ggl_column] = ds_fb_ggl[ggl_column].replace('',np.nan, regex=True)
    ds_fb_ggl[ggl_column].fillna(ds_fb_ggl[fb_column], inplace=True)

fill_missing_values('fb_country_name', 'ggl_country_name')
fill_missing_values('fb_region_name', 'ggl_region_name')
fill_missing_values('fb_city', 'ggl_city')
fill_missing_values('fb_phone', 'ggl_phone')

for i in range(ds_fb_ggl.shape[0]):
    if ds_fb_ggl.iloc[i].at['fb_categories'] == ['']:
        ds_fb_ggl.iloc[i].at['fb_categories'].remove('') 
        ds_fb_ggl.iloc[i].at['fb_categories'].append(ds_fb_ggl.iloc[i].at['ggl_category'])


Now let's see the results: 

In [29]:
ds_fb_ggl['company_match'] = (ds_fb_ggl['ggl_name'] == ds_fb_ggl['fb_name'])

ds_fb_ggl['country_match'] = (ds_fb_ggl['fb_country_name'] == ds_fb_ggl['ggl_country_name'])

ds_fb_ggl['region_match'] =(ds_fb_ggl['fb_region_name'] == ds_fb_ggl['ggl_region_name'])

ds_fb_ggl['phone_match'] = (ds_fb_ggl['fb_phone'] == ds_fb_ggl['ggl_phone'])

ds_fb_ggl['city_match'] = (ds_fb_ggl['fb_city'] == ds_fb_ggl['ggl_city'])

true = 0
for i in range(ds_fb_ggl.shape[0]):
    row = ds_fb_ggl.iloc[i]
    for item in row['fb_categories']:
        if row['ggl_category'] == item:
            true += 1
            
data = [['Accuracy', (ds_fb_ggl['company_match'].value_counts()/ds_fb_ggl.shape[0]*100).iloc[0], 
         (np.round(ds_fb_ggl['country_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0],
         (np.round(ds_fb_ggl['region_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
         (np.round(ds_fb_ggl['city_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
         (np.round(ds_fb_ggl['phone_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
          np.round(true/ds_fb_ggl.shape[0]*100,2)]]
table = tabulate(data, headers=['','Company Match', 'Country Match', 'Region Match', 'City Match', 'Phone Match', 'Category Match'], tablefmt='orgtbl')
print(table)

|          |   Company Match |   Country Match |   Region Match |   City Match |   Phone Match |   Category Match |
|----------+-----------------+-----------------+----------------+--------------+---------------+------------------|
| Accuracy |             100 |           98.84 |          96.91 |        90.83 |         90.41 |            54.62 |


Looks much better. Still, the accuracy in the category column is pretty low. Now I will start eliminating rows so that the accuracy gets to 100% in the columns.

In [30]:
df = ds_fb_ggl.drop(ds_fb_ggl[ds_fb_ggl['fb_country_name'] != ds_fb_ggl['ggl_country_name']].index)
print((df['fb_country_name'] == df['ggl_country_name']).value_counts())
df = df.drop(df[df['fb_region_name'] != df['ggl_region_name']].index)
print((df['fb_region_name'] == df['ggl_region_name']).value_counts())
df = df.drop(df[df['fb_city'] != df['ggl_city']].index)
print((df['fb_city'] == df['ggl_city']).value_counts())
df = df.drop(df[df['fb_phone'] != df['ggl_phone']].index)
print((df['fb_phone'] == df['ggl_phone']).value_counts())
df = df.reset_index()
ds_fb_ggl = df

True    42352
dtype: int64
True    41504
dtype: int64
True    38844
dtype: int64
True    36000
dtype: int64


In [31]:
ds_fb_ggl['company_match'] = (ds_fb_ggl['ggl_name'] == ds_fb_ggl['fb_name'])

ds_fb_ggl['country_match'] = (ds_fb_ggl['fb_country_name'] == ds_fb_ggl['ggl_country_name'])

ds_fb_ggl['region_match'] =(ds_fb_ggl['fb_region_name'] == ds_fb_ggl['ggl_region_name'])

ds_fb_ggl['phone_match'] = (ds_fb_ggl['fb_phone'] == ds_fb_ggl['ggl_phone'])

ds_fb_ggl['city_match'] = (ds_fb_ggl['fb_city'] == ds_fb_ggl['ggl_city'])

true = 0
for i in range(ds_fb_ggl.shape[0]):
    row = ds_fb_ggl.iloc[i]
    for item in row['fb_categories']:
        if row['ggl_category'] == item:
            true += 1

        
data = [['Accuracy', (ds_fb_ggl['company_match'].value_counts()/ds_fb_ggl.shape[0]*100).iloc[0], 
         (np.round(ds_fb_ggl['country_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0],
         (np.round(ds_fb_ggl['region_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
         (np.round(ds_fb_ggl['city_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
         (np.round(ds_fb_ggl['phone_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
          np.round(true/ds_fb_ggl.shape[0]*100,2)]]
table = tabulate(data, headers=['','Company Match', 'Country Match', 'Region Match', 'City Match', 'Phone Match', 'Category Match'], tablefmt='orgtbl')
print(table)

|          |   Company Match |   Country Match |   Region Match |   City Match |   Phone Match |   Category Match |
|----------+-----------------+-----------------+----------------+--------------+---------------+------------------|
| Accuracy |             100 |             100 |            100 |          100 |           100 |            55.04 |


In [32]:
ds_fb_ggl.to_csv('facebook_google_dataset.csv')

In [33]:
df = ds_fb_ggl
a =[]
for i in range(df.shape[0]):
    row = df.iloc[i]
    c = 0
    for item in row['fb_categories']:
        if row['ggl_category'] == item: 
            c += 1
    if c==0: 
        a.append(i)

In [34]:
print(len(a))
df = df.drop(a)
df = df.reset_index()
print(df.shape)

17799
(18201, 38)


**Final results for the Facebook - Google dataset:**

In [35]:
ds_fb_ggl = df
ds_fb_ggl['company_match'] = (ds_fb_ggl['ggl_name'] == ds_fb_ggl['fb_name'])

ds_fb_ggl['country_match'] = (ds_fb_ggl['fb_country_name'] == ds_fb_ggl['ggl_country_name'])

ds_fb_ggl['region_match'] =(ds_fb_ggl['fb_region_name'] == ds_fb_ggl['ggl_region_name'])

ds_fb_ggl['phone_match'] = (ds_fb_ggl['fb_phone'] == ds_fb_ggl['ggl_phone'])

ds_fb_ggl['city_match'] = (ds_fb_ggl['fb_city'] == ds_fb_ggl['ggl_city'])

true = 0
for i in range(ds_fb_ggl.shape[0]):
    item1 = ds_fb_ggl.iloc[i].at['fb_categories']
    item2 = ds_fb_ggl.iloc[i].at['ggl_category']
    if [value for value in item1 if value in item2]: true += 1
        
data = [['Accuracy', (ds_fb_ggl['company_match'].value_counts()/ds_fb_ggl.shape[0]*100).iloc[0], 
         (np.round(ds_fb_ggl['country_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0],
         (np.round(ds_fb_ggl['region_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
         (np.round(ds_fb_ggl['city_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
         (np.round(ds_fb_ggl['phone_match'].value_counts()/ds_fb_ggl.shape[0]*100,2)).iloc[0], 
          np.round(true/ds_fb_ggl.shape[0]*100,2)]]
table = tabulate(data, headers=['','Company Match', 'Country Match', 'Region Match', 'City Match', 'Phone Match', 'Category Match'], tablefmt='orgtbl')
print(table)

|          |   Company Match |   Country Match |   Region Match |   City Match |   Phone Match |   Category Match |
|----------+-----------------+-----------------+----------------+--------------+---------------+------------------|
| Accuracy |             100 |             100 |            100 |          100 |           100 |              100 |


This is the largest accurate dataset I could get, with ~18000 companies. I will merge this dataset with the website one.

In [36]:
del ds_fb_ggl['level_0']
del ds_fb_ggl['index']
ds_fb_ggl.head(15)

,fb_domain,fb_address,fb_categories,fb_city,fb_country_code,fb_country_name,fb_description,fb_email,fb_link,fb_name,...,ggl_region_code,ggl_region_name,ggl_text,ggl_zip_code,ggl_domain,company_match,country_match,region_match,phone_match,city_match
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",[Pest Services & Agricultural Chemicals],calvisson,fr,france,,,https://euro-hygiene-34.fr,euro hygiène,...,occ,occitanie,4.7 (3) · Pest control service 3+ years in bus...,,euro-hygiene-34.fr,True,True,True,True,True
1,lakesidehomeservices.ca,,"[Appliance Repair & Maintenance, Home Builders...",,ca,,,,https://lakesidehomeservices.ca,lakeside home services,...,,,5.0 (2) · Appliance repair service Closed ⋅ Op...,,lakesidehomeservices.ca,True,True,True,True,True
2,rossiterboats.com,,"[Boats & Yachts Dealers, Boat Tours & Cruises]",markdale,,canada,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,rossiter boats,...,on,ontario,"4.5 (4) · Boat builders Markdale, ON, Canada O...",,rossiterboats.com,True,True,True,True,True
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...","[Architects & Architectural Services, Other En...",vancouver,ca,canada,,office@caa-arcitecture.com,http://caa-architecture.com,chandler associates architecture,...,bc,british columbia,4.5 (2) · Architect 5+ years in business · Van...,v6c 2g8,caa-architecture.com,True,True,True,True,True
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...","[Insurance - Agents, Carriers & Brokers]",winnipeg,ca,canada,,,https://apexsurety.ca,apex surety & insurance,...,mb,manitoba,No reviews · Insurance broker 4 Donald St #3 O...,r3l 2t7,apexsurety.ca,True,True,True,True,True
5,dovercheese.com,"307 main street, n0a 1n0, port dover, on, cana...",[Dairy Products - Farms & Stores],port dover,ca,canada,The Dover Cheese Shop is a food lover’s paradi...,info@dovercheese.com,http://dovercheese.com,the dover cheese shop,...,on,ontario,"Cheese shop Port Dover, ON, Canada · +1 519-71...",n0a 1n0,dovercheese.com,True,True,True,True,True
6,alphascientific.ca,"8623 granville street unit 143, v6p 5a2, vanco...",[Medical Supply Manufacturers],vancouver,ca,canada,www.alphascientific.ca Alpha Scientific Repair...,sales@alphascientific.ca,http://alphascientific.ca,alpha scientific repair services,...,bc,british columbia,No reviews · Medical equipment supplier Vancou...,v6p 5a3,alphascientific.ca,True,True,True,True,True
7,brokenspirits.ca,"#4 – 3320 14 avenue ne, t2a 6j4, calgary, ab, ...",[Distilleries & Malt Processing],calgary,ca,canada,,,https://brokenspirits.ca,broken spirits distillery,...,ab,alberta,5.0 (8) · Distillery 3320 14 Ave NE Unit 4 Clo...,t2a 6j4,brokenspirits.ca,True,True,True,True,True
8,marilees.com,"170 hwy 20 w unit 5, l0s1e0, fonthill, on, can...",[Bridal Shops & Wedding Supplies],fonthill,ca,canada,Marilee's Bridal - Niagara's favourite boutiqu...,info@marilees.com,http://marilees.com,marilee's bridal,...,on,ontario,4.5 (73) · Bridal shop 5+ years in business · ...,l0s 1e5,marilees.com,True,True,True,True,True
9,albertashrinkwrap.com,"rr1, t0c 0j0, bentley, ab, canada, alberta",[Construction Services],,ca,,,,https://albertashrinkwrap.com,alberta shrink wrap,...,,,"5.0 (1) · Construction company Bentley, AB, Ca...",,albertashrinkwrap.com,True,True,True,True,True


**<font size="4"> MERGING ALL THE THREE DATASETS (AGAIN) </font>**

In [37]:
ds_all = ds_fb_ggl.merge(web_ds, left_on='ggl_name', right_on='web_legal_name')
ds_all.head()

,fb_domain,fb_address,fb_categories,fb_city,fb_country_code,fb_country_name,fb_description,fb_email,fb_link,fb_name,...,web_domain_suffix,web_language,web_legal_name,web_main_city,web_main_country,web_main_region,web_phone,web_site_name,web_tld,web_category
0,lakesidehomeservices.ca,,"[Appliance Repair & Maintenance, Home Builders...",,ca,,,,https://lakesidehomeservices.ca,lakeside home services,...,ca,en,lakeside home services,stittsville,canada,ontario,16138318840,Lakeside Home Services,ca,Appliance Repair & Maintenance
1,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...","[Architects & Architectural Services, Other En...",vancouver,ca,canada,,office@caa-arcitecture.com,http://caa-architecture.com,chandler associates architecture,...,com,en,chandler associates architecture,vancouver,canada,british columbia,16046873325,Chandler,com,Architects & Architectural Services
2,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...","[Insurance - Agents, Carriers & Brokers]",winnipeg,ca,canada,,,https://apexsurety.ca,apex surety & insurance,...,ca,en,apex surety & insurance,winnipeg,canada,manitoba,12045602508,INSURANCEAPEX,ca,Liability Insurance Agency
3,alphascientific.ca,"8623 granville street unit 143, v6p 5a2, vanco...",[Medical Supply Manufacturers],vancouver,ca,canada,www.alphascientific.ca Alpha Scientific Repair...,sales@alphascientific.ca,http://alphascientific.ca,alpha scientific repair services,...,ca,en,alpha scientific repair services,vancouver,canada,british columbia,16046903380,Alpha Scientific,ca,Medical Supply Manufacturers
4,parksiderentals.ca,"birds hill park hwy 206, r0e 0k0, winnipeg, mb...","[Boat Rental & Chartering Services, Boat Tours...",winnipeg,ca,canada,All rentals come with a trailer so you can go ...,,http://parksiderentals.ca,parkside rentals,...,ca,en,parkside rentals,dugald,canada,manitoba,12042325434,Parkside Rentals,ca,Boat Rental & Chartering Services


In [38]:
# ds = ds_all
ds_all['company_match'] = ((ds_all['web_legal_name'] == ds_all['ggl_name'])&(ds_all['ggl_name'] == ds_all['fb_name']))

ds_all['country_match'] = ((ds_all['web_main_country'] == ds_all['fb_country_name'])&(ds_all['fb_country_name'] == ds_all['ggl_country_name']))

ds_all['region_match'] = ((ds_all['web_main_region'] == ds_all['fb_region_name'])&(ds_all['fb_region_name'] == ds_all['ggl_region_name']))

ds_all['phone_match'] = ((ds_all['web_phone'] == ds_all['fb_phone'])&(ds_all['fb_phone'] == ds_all['ggl_phone']))

ds_all['city_match'] = ((ds_all['web_main_city'] == ds_all['fb_city']) & (ds_all['fb_city'] == ds_all['ggl_city']))

true = 0
for i in range(ds_all.shape[0]):
    row = ds_all.iloc[i]
    for item in row['fb_categories']:
        if row['ggl_category'] == item and row['web_category'] == item:
            true += 1

data = [['Accuracy', (ds_all['company_match'].value_counts()/ds_all.shape[0]*100).iloc[0], (np.round(ds_all['country_match'].value_counts()/ds_all.shape[0]*100,2)).iloc[0],
     (np.round(ds_all['region_match'].value_counts()/ds_all.shape[0]*100,2)).iloc[0], (np.round(ds_all['city_match'].value_counts()/ds_all.shape[0]*100,2)).iloc[0],
         (np.round(ds_all['phone_match'].value_counts()/ds_all.shape[0]*100,2)).iloc[0], np.round(true/ds_all.shape[0]*100,2)]]
table = tabulate(data, headers=['','Company Match', 'Country Match', 'Region Match', 'City Match', 'Phone Match', 'Category Match'], tablefmt='orgtbl')
print(table)

|          |   Company Match |   Country Match |   Region Match |   City Match |   Phone Match |   Category Match |
|----------+-----------------+-----------------+----------------+--------------+---------------+------------------|
| Accuracy |             100 |           80.62 |           78.2 |        66.84 |         75.84 |            54.86 |


This is better than the first dataset. I will now perform the same steps in order to get a better accuracy.

In [39]:
def fill_missing_values(fb_column, web_column):
#     ds_all[fb_column] = ds_all[fb_column].replace('',np.nan, regex=True)
#     ds_all[fb_column].fillna(ds_all[web_column], inplace=True)
    ds_all[web_column] = ds_all[web_column].replace('',np.nan, regex=True)
    ds_all[web_column].fillna(ds_all[fb_column], inplace=True)

fill_missing_values('fb_country_name', 'web_main_country')
fill_missing_values('fb_region_name', 'web_main_region')
fill_missing_values('fb_city', 'web_main_city')
fill_missing_values('fb_phone', 'web_phone')


In [40]:
df = ds_all
df = df.drop(df[df['fb_country_name'] != df['web_main_country']].index)
print((df['fb_country_name'] == df['web_main_country']).value_counts())
df = df.drop(df[df['fb_region_name'] != df['web_main_region']].index)
print((df['fb_region_name'] == df['web_main_region']).value_counts())
df = df.drop(df[df['fb_city'] != df['web_main_city']].index)
print((df['fb_city'] == df['web_main_city']).value_counts())
df = df.drop(df[df['fb_phone'] != df['web_phone']].index)
print((df['fb_phone'] == df['web_phone']).value_counts())
df = df.reset_index()

True    4527
dtype: int64
True    4362
dtype: int64
True    3762
dtype: int64
True    3114
dtype: int64


In [41]:
ds_all = df
a =[]
for i in range(ds_all.shape[0]):
    row = ds_all.iloc[i]
    c = 0
    for item in row['fb_categories']:
        if row['ggl_category'] == item and row['web_category'] == item: 
            c += 1
    if c==0: 
        a.append(i)
ds_all = ds_all.drop(a)
ds_all = ds_all.reset_index()

In [42]:
# ds = ds_all
ds_all['company_match'] = ((ds_all['web_legal_name'] == ds_all['ggl_name']) & (ds_all['ggl_name'] == ds_all['fb_name']))

ds_all['country_match'] = ((ds_all['web_main_country'] == ds_all['fb_country_name'])&(ds_all['fb_country_name'] == ds_all['ggl_country_name']))

ds_all['region_match'] = ((ds_all['web_main_region'] == ds_all['fb_region_name'])&(ds_all['fb_region_name'] == ds_all['ggl_region_name']))

ds_all['phone_match'] = ((ds_all['web_phone'] == ds_all['fb_phone'])&(ds_all['fb_phone'] == ds_all['ggl_phone']))

ds_all['city_match'] = ((ds_all['web_main_city'] == ds_all['fb_city']) & (ds_all['fb_city'] == ds_all['ggl_city']))

true = 0
for i in range(ds_all.shape[0]):
    item1 = ds_all.iloc[i].at['fb_categories']
    item2 = ds_all.iloc[i].at['ggl_category']
    item3 = ds_all.iloc[i].at['web_category']
    c =  [value for value in item1 if value in item2]
    if [value for value in c if value in item3]: true += 1

data = [['Accuracy', (ds_all['company_match'].value_counts()/ds_all.shape[0]*100).iloc[0], (np.round(ds_all['country_match'].value_counts()/ds_all.shape[0]*100,2)).iloc[0],
     (np.round(ds_all['region_match'].value_counts()/ds_all.shape[0]*100,2)).iloc[0], (np.round(ds_all['city_match'].value_counts()/ds_all.shape[0]*100,2)).iloc[0],
         (np.round(ds_all['phone_match'].value_counts()/ds_all.shape[0]*100,2)).iloc[0], np.round(true/ds_all.shape[0]*100,2)]]
table = tabulate(data, headers=['','Company Match', 'Country Match', 'Region Match', 'City Match', 'Phone Match', 'Category Match'], tablefmt='orgtbl')
print(table)

|          |   Company Match |   Country Match |   Region Match |   City Match |   Phone Match |   Category Match |
|----------+-----------------+-----------------+----------------+--------------+---------------+------------------|
| Accuracy |             100 |             100 |            100 |          100 |           100 |              100 |


In [43]:
del ds_all['level_0']
del ds_all['index']

In [44]:
ds_all.to_csv('complete_dataset.csv')

In [45]:
print(ds_all.shape)

(1610, 47)


So I merged all the three datasets available and got around 1600 companies. This is the complete dataset which features all the 3 datasets merged. 